In [1]:
from model import Net
import torch
import config
import onnxruntime as ort
import numpy as np

# Load the PyTorch Model

In [2]:
model = Net()
model.load_state_dict(torch.load(config.MODEL_PATH, map_location=torch.device('cpu')))

<All keys matched successfully>

 - https://learn.microsoft.com/en-us/windows/ai/windows-ml/tutorials/pytorch-convert-model
 
 - It's important to call model.eval()  before exporting the model, as this sets the model to inference mode. 
 - This is needed since operators like dropout or batchnorm behave differently in inference and training mode.

In [3]:

model.eval()

Net(
  (conv_1): Conv2d(3, 128, kernel_size=(6, 6), stride=(1, 1), padding=(1, 1))
  (pool_1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv_2): Conv2d(128, 64, kernel_size=(6, 6), stride=(2, 2), padding=(1, 1))
  (pool_2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv_3): Conv2d(64, 32, kernel_size=(6, 6), stride=(2, 2), padding=(1, 1))
  (pool_3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (drop_1): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=2048, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (drop_2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=60, out_features=1, bias=True)
)

# Export to ONNX format
- https://pytorch.org/docs/stable/onnx.html#example-alexnet-from-pytorch-to-onnx

 - In onnx export we should we give an expected dummy input or example input by model, 
 - in pytorch the input image array to the model should be in format NCHW
 - batch N, channels C, height H, width W.
 - since we are making prediction only for one image at time in an api request
 - batch size is kept 1

In [4]:
channels = 3
dummy_input  = torch.randn(config.BATCH_SIZE, channels, config.IMAGE_HEIGHT, config.IMAGE_WIDTH)
dummy_input.shape

torch.Size([1, 3, 300, 300])

- https://pytorch.org/docs/stable/onnx.html#example-alexnet-from-pytorch-to-onnx

- Providing input (`actual_input`) and output names (`output`) sets the display names for values within the model's graph. 
- Setting these does not change the semantics of the graph; it is only for readability.


In [5]:
torch.onnx.export(
    model,
    dummy_input,
    "model.onnx",
    input_names=["actual_input"],
    output_names=["output"])

- we can view the model architecture by uploading onnx file to 
- https://netron.app/


# Test the onnx format


- After the model is saved to onnx format we dont need pytorch or anyother deeplearning frameworks to make inference
- https://onnxruntime.ai/docs/api/python/api_summary.html#inferencesession

In [7]:
# generate a random input
test_input = np.random.randn(1, 3, 300, 300).astype(np.float32)

In [8]:
ort_session = ort.InferenceSession("model.onnx")

In [9]:
outputs = ort_session.run(None, {"actual_input": test_input})

In [10]:
outputs

[array([[-1.9428433]], dtype=float32)]

 - At inference we will using our custom sigmoid function, since we are not using torch

In [11]:
def get_sigmoid(x):
    """
    to apply sigmoid
    """
    return 1/(1 + np.exp(-x))

In [12]:
get_sigmoid(outputs[0].flatten()[0])

0.12533582576367583